In [1]:
n = 4

a = 6
g = 10
number = 11

bits = [ int(x) for x in '{:0{size}b}'.format(number, size=n) ]

In [2]:
bits

[1, 0, 1, 1]

In [3]:
for i in range(n - 1, 0, -1):
    print(bits[n-(i+1)])

1
0
1


In [4]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [5]:
reg_A = QuantumRegister(n, name='a')
reg_G = QuantumRegister(n, name='g')
reg_C = QuantumRegister(1, name='c')

In [6]:
qc = QuantumCircuit(reg_A, reg_G, reg_C)

qc.cx(reg_G[n - 1], reg_C)

for i in range(n - 1, 0, -1):
    if bits[n-(i+1)] == 1:
        qc.cx(reg_A[i], reg_G[i])
        qc.x(reg_A[i])
    qc.ccx(reg_G[i - 1], reg_A[i], reg_G[i])
    # qc.barrier(reg_A[:], reg_G[:], reg_C[:])
    
if bits[n-1] == 1:
    qc.cx(reg_A[0], reg_G[0])
    
for i in range(1, n):
    qc.ccx(reg_G[i - 1], reg_A[i], reg_G[i])

#qc.barrier(reg_A[:], reg_G[:], reg_C[:])
    
qc.cx(reg_G[n - 1], reg_C)

#qc.barrier(reg_A[:], reg_G[:], reg_C[:])

for i in range(n - 1, 0, -1):
    qc.ccx(reg_G[i - 1], reg_A[i], reg_G[i])

#qc.barrier(reg_A[:], reg_G[:], reg_C[:])

if bits[n-1] == 1:
    qc.cx(reg_A[0], reg_G[0])

#qc.barrier(reg_A[:], reg_G[:], reg_C[:])
    
for i in range(1, n):
    qc.ccx(reg_G[i - 1], reg_A[i], reg_G[i])
    if bits[n-(i+1)] == 1:
        qc.x(reg_A[i])
        qc.cx(reg_A[i], reg_G[i])
    #qc.barrier(reg_A[:], reg_G[:], reg_C[:])

In [7]:
# qc.draw(output='mpl')

In [8]:
cr0 = ClassicalRegister(n, name='ca')
cr1 = ClassicalRegister(n, name='cg')
cr2 = ClassicalRegister(1, name='carry')

qc_meas = QuantumCircuit(reg_A, reg_G, reg_C, cr0, cr1, cr2)
qc_meas.barrier(reg_A[:], reg_G[:], reg_C[:])
qc_meas.measure(reg_A[:], cr0[:])
qc_meas.measure(reg_G[:], cr1[:])
qc_meas.measure(reg_C[:], cr2[:])

# qc_meas.draw(output='mpl')

In [9]:
qc_init = QuantumCircuit(reg_A, reg_G)

a_bits = [ int(x) for x in '{:0{size}b}'.format(a, size=n) ]
g_bits = [ int(x) for x in '{:0{size}b}'.format(g, size=n) ]

for i in range(n):
    if a_bits[n-(i+1)] == 1:
        qc_init.x(reg_A[i])
        
for i in range(n):
    if g_bits[n-(i+1)] == 1:
        qc_init.x(reg_G[i])

qc_init.barrier(reg_A[:], reg_G[:])
        
# qc_init.draw(output='mpl')

In [10]:
qc_all = qc_init + qc + qc_meas
# qc_all.draw(output='mpl')

<ipython-input-10-7badd2fe3392>:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  qc_all = qc_init + qc + qc_meas
/home/bartek/S10/MGR/masterThesisProject/venv/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:869: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return self.combine(rhs)


In [11]:
from qiskit import Aer
from qiskit import execute

backend_sim = Aer.get_backend('qasm_simulator')
job_sim = execute(qc_all, backend_sim, shots=1024)

result_sim = job_sim.result()

In [12]:
counts = result_sim.get_counts(qc_all)
print(counts)

{'1 1010 0110': 1024}
